In [18]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm
import tensorflow as tf
from tensorflow import keras
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from sklearn.metrics import accuracy_score
from keras.applications.xception import Xception

In [3]:
DATADIR = 'archive'
SUBDIRS = ["train", "test"]
CATEGORIES = ["american_football","billiard_ball","bowling_ball","cricket_ball","football","golf_ball","hockey_ball","hockey_puck","rugby_ball","shuttlecock","table_tennis_ball","tennis_ball","volleyball","baseball", "basketball"]

dataset = {
    "train": [],
    "test": []
}


for subdir in SUBDIRS:
    x = 0
    for category in CATEGORIES: 
        path = os.path.join(DATADIR, subdir, category)
        for img in tqdm(os.listdir(path)):
            img_array = cv2.imread(os.path.join(path, img)).astype(np.float32)/255.0
            IMG_SIZE = 150
            new_array = cv2.resize(img_array, (IMG_SIZE,IMG_SIZE))
            new_array_rgb = cv2.cvtColor(new_array, cv2.COLOR_BGR2RGB)
            dataset[subdir].append([new_array_rgb,x])

        x += 1

            
            
            
            


100%|██████████| 86/86 [00:01<00:00, 68.45it/s]


In [4]:
print (img_array)
print (img_array.shape)

[[[0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  ...
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]]

 [[0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  ...
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]]

 [[0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  ...
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]]

 ...

 [[0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  ...
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]]

 [[0.9490196  0.92941177 0.9254902 ]
  [0.9490196  0.92941177 0.9254902 ]


In [5]:
Xtrain = []
Ytrain = []

for features, label in dataset["train"]: 
    Xtrain.append(features)
    Ytrain.append(label)

Xtrain = np.array(Xtrain).reshape(-1, IMG_SIZE,IMG_SIZE, 3)
y = np.array(Ytrain)

In [6]:
model = keras.Sequential(
    [
        Conv2D(32, (4, 4), activation="relu", input_shape=(IMG_SIZE, IMG_SIZE, 3)),
        MaxPooling2D((2, 2)),
        Conv2D(64, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Conv2D(128, (3, 3), activation="relu"),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(512, activation="relu"),
        Dense(256, activation="relu"),
        Dense(128, activation="relu"),
        Dense(64, activation="relu"),
        Dense(64, activation="relu"),
        Dense(32, activation="relu"),
        Dense(15, activation="softmax")
    ]
)
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])


c:\Users\youjo\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.fit(Xtrain,y, epochs = 5)

Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 52s 224ms/step - accuracy: 0.0956 - loss: 2.6647
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 50s 220ms/step - accuracy: 0.1639 - loss: 2.5144
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 51s 223ms/step - accuracy: 0.2364 - loss: 2.3710
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 49s 214ms/step - accuracy: 0.3150 - loss: 2.1159
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 49s 213ms/step - accuracy: 0.4121 - loss: 1.8310


In [9]:
def load_data(data_dir):
    data = []
    for category in CATEGORIES: 
        path = os.path.join(data_dir, category)
        class_num = CATEGORIES.index(category)
        for img in tqdm(os.listdir(path)): 
            try: 
                img_array = cv2.imread(os.path.join(path, img))
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                data.append([new_array, class_num])
            except Exception as e: 
                pass
            
    return data


train_data = load_data(os.path.join(DATADIR, 'train'))
test_data = load_data(os.path.join(DATADIR, 'test'))


X_train = np.array([i[0] for i in train_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_train = np.array([i[1] for i in train_data])

X_test = np.array([i[0] for i in test_data]).reshape(-1, IMG_SIZE, IMG_SIZE, 3)
y_test = np.array([i[1] for i in test_data])


model.fit(X_train, y_train, epochs=5)


model.evaluate(X_test, y_test)


  0%|          | 0/384 [00:00<?, ?it/s]

100%|██████████| 86/86 [00:00<00:00, 115.46it/s]


Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 68s 291ms/step - accuracy: 0.0920 - loss: 17.0622
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 66s 288ms/step - accuracy: 0.0855 - loss: 2.6822
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 65s 286ms/step - accuracy: 0.1171 - loss: 2.6554
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 65s 284ms/step - accuracy: 0.1192 - loss: 2.6303
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 65s 282ms/step - accuracy: 0.1375 - loss: 2.5960
58/58 ━━━━━━━━━━━━━━━━━━━━ 4s 67ms/step - accuracy: 0.1830 - loss: 2.5729


[2.6307003498077393, 0.11732754111289978]

In [10]:
prediction = model.predict(X_test)

58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step


In [12]:
pred = np.argmax(prediction, axis=1)

In [16]:
accuracy = accuracy_score(y_test, pred)
print ("Accuracy: ", accuracy)

Accuracy:  0.11732753938077133


In [17]:
#Model 2
keras.applications.Xception(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

91884032/91884032 ━━━━━━━━━━━━━━━━━━━━ 50s 1us/step


<Functional name=xception, built=True>

In [20]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 63s 264ms/step - accuracy: 0.1495 - loss: 2.5517
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 66s 286ms/step - accuracy: 0.1854 - loss: 2.4550
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 62s 270ms/step - accuracy: 0.2163 - loss: 2.3857
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 60s 263ms/step - accuracy: 0.2627 - loss: 2.2698
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 55s 241ms/step - accuracy: 0.2996 - loss: 2.1420


In [23]:
prediction = model.predict(X_test)
pred = np.argmax(prediction, axis=1)
accuracy = accuracy_score(y_test, pred)
print ("Accuracy: ", accuracy)

58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 50ms/step
Accuracy:  0.17110266159695817


In [25]:
#Model 3
keras.applications.InceptionV3(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

96112376/96112376 ━━━━━━━━━━━━━━━━━━━━ 49s 1us/step


<Functional name=inception_v3, built=True>

In [26]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 61s 255ms/step - accuracy: 0.3499 - loss: 1.9875
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 60s 263ms/step - accuracy: 0.4090 - loss: 1.8140
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 64s 278ms/step - accuracy: 0.4477 - loss: 1.6768
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 64s 279ms/step - accuracy: 0.4807 - loss: 1.5908
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 64s 281ms/step - accuracy: 0.5215 - loss: 1.4779


In [27]:
prediction = model.predict(X_test)
pred = np.argmax(prediction, axis=1)
accuracy = accuracy_score(y_test, pred)
print ("Accuracy: ", accuracy)

58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
Accuracy:  0.21455730581205867


In [28]:
#Model 4
keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

553467096/553467096 ━━━━━━━━━━━━━━━━━━━━ 302s 1us/step


<Functional name=vgg16, built=True>

In [29]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 54s 218ms/step - accuracy: 0.5417 - loss: 1.4152
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 50s 217ms/step - accuracy: 0.5831 - loss: 1.2684
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 50s 217ms/step - accuracy: 0.6124 - loss: 1.2156
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 50s 218ms/step - accuracy: 0.6108 - loss: 1.2083
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 55s 238ms/step - accuracy: 0.6153 - loss: 1.2023


In [30]:
prediction = model.predict(X_test)
pred = np.argmax(prediction, axis=1)
accuracy = accuracy_score(y_test, pred)
print ("Accuracy: ", accuracy)

58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step
Accuracy:  0.24932102118413904


In [31]:
#Model 5
keras.applications.MobileNet(
    input_shape=None,
    alpha=1.0,
    depth_multiplier=1,
    dropout=0.001,
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
)

17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 9s 1us/step


<Functional name=mobilenet_1.00_224, built=True>

In [32]:
model.compile(optimizer='adam',loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=5)

Epoch 1/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 54s 228ms/step - accuracy: 0.6446 - loss: 1.0918
Epoch 2/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 51s 221ms/step - accuracy: 0.6631 - loss: 1.0531
Epoch 3/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 50s 217ms/step - accuracy: 0.6849 - loss: 0.9947
Epoch 4/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 61s 266ms/step - accuracy: 0.6935 - loss: 0.9658
Epoch 5/5
229/229 ━━━━━━━━━━━━━━━━━━━━ 53s 231ms/step - accuracy: 0.7074 - loss: 0.9185


In [33]:
prediction = model.predict(X_test)
pred = np.argmax(prediction, axis=1)
accuracy = accuracy_score(y_test, pred)
print ("Accuracy: ", accuracy)

58/58 ━━━━━━━━━━━━━━━━━━━━ 3s 54ms/step
Accuracy:  0.265073329712113
